In [1]:
import ipywidgets as widgets
from IPython.display import display

# Create a simple slider widget
slider = widgets.IntSlider(value=10, min=0, max=100, step=1, description='Test Slider:')
display(slider)


IntSlider(value=10, description='Test Slider:')

In [ ]:
from PIL import Image, ImageDraw
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
def generate_line_pattern(line_width_um=10, spacing_um=10, length_mm=24, span_mm=72, resolution=14):
    """
    Generates a PNG image with horizontal lines based on the specified parameters.

    Parameters:
    - line_width_um: Width of each line in micrometers.
    - spacing_um: Spacing between lines in micrometers.
    - length_mm: Length of each line in millimeters.
    - span_mm: Total span of the image in millimeters.
    - resolution: Pixels per 10 micrometers.

    Returns:
    - PIL Image object.
    """
    try:
        # Convert measurements to pixels
        line_width_px = max(1, int((line_width_um / 10) * resolution))
        spacing_px = max(1, int((spacing_um / 10) * resolution))
        length_px = int((length_mm * 1000 / 10) * resolution)  # 1 mm = 1000 µm
        span_px = int((span_mm * 1000 / 10) * resolution)

        # Calculate number of lines
        lines = span_px // (line_width_px + spacing_px)

        # Calculate image size
        img_width = length_px
        img_height = lines * (line_width_px + spacing_px)
        img_height = min(img_height, span_px)  # Ensure it doesn't exceed span_px

        # Safety check for image size
        max_pixels = 10000 * 10000  # Example limit: 100 million pixels
        if img_width * img_height > max_pixels:
            raise ValueError(f"Image size {img_width}x{img_height} px is too large. Reduce parameters or resolution.")

        # Create image
        image = Image.new('RGB', (img_width, img_height), 'white')
        draw = ImageDraw.Draw(image)

        for i in range(lines):
            y_start = i * (line_width_px + spacing_px)
            y_end = y_start + line_width_px
            if y_start >= img_height:
                break
            y_end = min(y_end, img_height)
            draw.rectangle([0, y_start, img_width, y_end], fill='black')

        return image

    except Exception as e:
        print(f"Error: {e}")
        return None
# Define widgets
line_width_slider = widgets.IntSlider(value=10, min=1, max=100, step=1, description='Line Width (µm):')
spacing_slider = widgets.IntSlider(value=10, min=1, max=100, step=1, description='Spacing (µm):')
length_input = widgets.FloatText(value=24.0, description='Length (mm):')
span_input = widgets.FloatText(value=72.0, description='Span (mm):')
resolution_slider = widgets.IntSlider(value=14, min=1, max=100, step=1, description='Resolution (px per 10µm):')

generate_button = widgets.Button(description="Generate PNG", button_style='success')

output = widgets.Output()

# Display widgets
display(line_width_slider, spacing_slider, length_input, span_input, resolution_slider, generate_button, output)
def on_generate_clicked(b):
    with output:
        clear_output()
        # Retrieve values from widgets
        line_width_um = line_width_slider.value
        spacing_um = spacing_slider.value
        length_mm = length_input.value
        span_mm = span_input.value
        resolution = resolution_slider.value

        # Generate image
        image = generate_line_pattern(line_width_um, spacing_um, length_mm, span_mm, resolution)

        if image:
            # Display image
            display(image)

            # Provide download link
            from IPython.display import FileLink
            image.save("line_pattern.png")
            display(FileLink("10_line_pattern.png"))

# Link the button to the callback
generate_button.on_click(on_generate_clicked)
